In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
input_file = "balanced_dataset_final.csv" 
output_file = "remapping_dataset.csv"
# Đọc bằng Dask cho file lớn
ddf = dd.read_csv(input_file)

# Remap labels
ddf['label'] = ddf['label'].map({0:0, 4:1}, meta=('label', 'int64'))

# Shuffle (Dask không hỗ trợ shuffle trực tiếp, cần workaround)
ddf = ddf.sample(frac=1, random_state=42)

# Lưu kết quả
with ProgressBar():
    ddf.to_csv(output_file, single_file=True, index=False) 

[########################################] | 100% Completed | 125.62 s


/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/dask/dataframe/core.py:8153: UserWarning: Insufficient elements for `head`. 11000000 elements requested, only 226716 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


In [5]:
"""
Kiểm tra dataset sau khi xử lý
"""
#df = dk.read_csv("single_file.csv")
    
# Tính tổng số mẫu
total_samples = ddf.shape[0].compute()
print(f"Tổng số mẫu: {total_samples}")
    
# Tính phân bố label
label_counts = ddf['label'].value_counts().compute()
print("Phân bố label:")
print(label_counts.sort_index())
    
# Kiểm tra số lượng label liên tiếp
df_small = ddf.head(11000000)
labels = df_small['label'].values
max_consecutive = current = 1  

for i in range(1, len(labels)):
    if labels[i] == labels[i-1]:
        current += 1
        max_consecutive = max(max_consecutive, current)
    else:
        current = 1  # Reset bộ đếm khi label thay đổi

print(f"Chuỗi liên tiếp dài nhất trong 11 triệu mẫu đầu: {max_consecutive}")

Tổng số mẫu: 11098198
Phân bố label:
label
0     1098195
1    10000003
Name: count, dtype: int64
Chuỗi liên tiếp dài nhất trong 11 triệu mẫu đầu: 226716


In [6]:
import numpy as np
import pandas as pd
import dask.dataframe as dk
import tensorflow as tf
file_path = "remapping_dataset.csv"
out_path =   "shuffled_remapping_df_2type/"

df = dk.read_csv(file_path, blocksize="10MB")
# df = df.drop(columns=['Variance', 'Weight'])
print(df.npartitions)
print(df.dtypes)
print(df.head(10))
from dask_ml.model_selection import train_test_split 
from sklearn.utils import shuffle

def shuffle_dask_dataframe(df, random_state=None):
    return (
        df.map_partitions(
            lambda part: part.assign(__shuffle_key=np.random.RandomState(random_state).random(len(part)))
        )
        .shuffle('__shuffle_key')
        .drop(columns='__shuffle_key')
    )
    
df = shuffle_dask_dataframe(df)

print(df.dtypes)
print(df.npartitions)
# print(df.head(20))
df.to_parquet(out_path, engine='pyarrow')

2025-05-31 10:13:37.022614: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-31 10:13:37.065315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748661217.078486     544 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748661217.082586     544 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748661217.102713     544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

3102
flow_duration      float64
Header_Length      float64
Protocol Type      float64
Duration           float64
Rate               float64
Srate              float64
Drate              float64
fin_flag_number    float64
syn_flag_number    float64
rst_flag_number    float64
psh_flag_number    float64
ack_flag_number    float64
ece_flag_number    float64
cwr_flag_number    float64
ack_count          float64
syn_count          float64
fin_count          float64
urg_count          float64
rst_count          float64
HTTP               float64
HTTPS              float64
DNS                float64
Telnet             float64
SMTP               float64
SSH                float64
IRC                float64
TCP                float64
UDP                float64
DHCP               float64
ARP                float64
ICMP               float64
IPv                float64
LLC                float64
Tot sum            float64
Min                float64
Max                float64
AVG                floa

KeyboardInterrupt: 